In [ ]:
# !pip install torch torchvision
!pip install datasets
!pip install transformers
!pip install --pre timm
!pip install wandb
!pip install opencv-python

In [2]:
from __future__ import print_function, division
import os
import torch
import timm
import pandas as pd
# from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch import nn
from torch.optim import Adam, AdamW

from datasets import Dataset
# from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torch.utils.data import DataLoader
import torch.optim as optim
from torch.optim import lr_scheduler
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm
import time
import copy
import cv2
import wandb
import uuid
import tempfile
from datetime import datetime, date

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode

In [3]:
CELL_PAINTING_DIR = '/home/ubuntu/src'
MODEL_OUTPUT_DIR = '/dev/shm'
CACHE_DIR = CELL_PAINTING_DIR + '/cache'
BROAD_DIR = CELL_PAINTING_DIR + '/data/cpg0019-moshkov-deepprofiler/broad'

In [4]:
class_labels_to_int = {
    'AKT1_E17K': 0,
    'AKT1_WT': 1,
    'BRAF_V600E': 2,
    'BRAF_WT': 3,
    'CDC42_Q61L': 4,
    'CDC42_T17N': 5,
    'CDC42_WT': 6,
    'EMPTY': 7,
    'KRAS_G12V': 8,
    'KRAS_WT': 9,
    'RAF1_L613V': 10,
    'RAF1_WT': 11,
    'RHOA_Q63L': 12,
    'RHOA_WT': 13
}
print(len(class_labels_to_int))

14


### Prepare train/dev dataset (Optional)

In [5]:
def add_image_paths(row, delim='../../', prefix=BROAD_DIR):
    _, path_suffix = row.Image_Name.split(delim)
    path = prefix + "/" + path_suffix
    return path

def do_the_thing(dataset_name: str):
    from data_fetcher import DataFetcher
    dataset = pd.read_parquet(CACHE_DIR + f"/{dataset_name}_metadata.parquet", engine="pyarrow")

    dataset['Path'] = dataset.apply(lambda x: add_image_paths(x), axis=1)
    dataset['Labels'] = dataset["Treatment"].replace(
          to_replace=class_labels_to_int)
    
    dataset_clean = DataFetcher.clean_data(dataset)
    DataFetcher.create_train_dev_split(dataset_clean, CACHE_DIR)

    # Add channel information to each train and dev dataset
    for dataset_t in ["train", "dev"]:
        dataset_file = CACHE_DIR + f"/{dataset_t}/data.parquet"
        dataset_df = pd.read_parquet(dataset_file, engine="pyarrow")
        dataset_df = DataFetcher.repeat_channels(dataset_df)
        dataset_df = dataset_df.drop(columns=['__index_level_0__'])
        dataset_df.to_parquet(dataset_file, engine="pyarrow")
        print(f"Added channels to {dataset_file}, shape is: {dataset_df.shape}")


In [6]:
do_the_thing("c037_27k")

Saved dev data with shape (5490, 17) to /home/ubuntu/src/cache/dev
Saved train data with shape (21958, 17) to /home/ubuntu/src/cache/train
Added channels to /home/ubuntu/src/cache/train/data.parquet, shape is: (109790, 17)
Added channels to /home/ubuntu/src/cache/dev/data.parquet, shape is: (27450, 17)


### Create a custom pytorch Dataset

### Check to see if custom Dataset is working

In [ ]:
# training_data = MaxVitDataset(CFG, "train")
# dev_data = MaxVitDataset(CFG, "dev")

In [ ]:
# print("Training and dev data sizes")
# print(len(training_data))
# print(len(dev_data))
# print("Training and dev data at idx")
# print(training_data[0][0].shape)
# print(training_data[0][0].dtype)
# print(training_data[0][0])
# print(training_data[0][1])

### Inspect a subset of images

In [ ]:
import random
from typing import List

# 1. Take in a Dataset as well as a list of class names
def display_random_images(dataset: torch.utils.data.dataset.Dataset,
                          classes: List[str] = None,
                          n: int = 10,
                          display_shape: bool = True,
                          seed: int = None):
    
    # 2. Adjust display if n too high
    if n > 10:
        n = 10
        display_shape = False
        print(f"For display purposes, n shouldn't be larger than 10, setting to 10 and removing shape display.")
    
    # 3. Set random seed
    if seed:
        random.seed(seed)

    # 4. Get random sample indexes
    random_samples_idx = random.sample(range(len(dataset)), k=n)

    # 5. Setup plot
    plt.figure(figsize=(16, 8))

    # 6. Loop through samples and display random samples 
    for i, targ_sample in enumerate(random_samples_idx):
        targ_image, targ_label = dataset[targ_sample][0], dataset[targ_sample][1]

        # 7. Adjust image tensor shape for plotting: [color_channels, height, width] -> [color_channels, height, width]
        targ_image_adjust = targ_image.permute(1, 2, 0)

        # Plot adjusted samples
        plt.subplot(1, n, i+1)
        plt.imshow(targ_image_adjust)
        plt.axis("off")
        title = f"class: {targ_label}"
        if display_shape:
            title = title + f"\nshape: {targ_image_adjust.shape}"
        plt.title(title)

In [ ]:
# display_random_images(training_data, n=5)

### Fine Tuning With Timm

In [ ]:
timm.list_models("**maxvit**", pretrained=True)

In [ ]:
class CFG:
  data_dir = CACHE_DIR
  debug = False
  n_gpu = 1
  # device = "cpu" # ['cpu', 'mps']
  img_size = 224
  ### total # of classes in this dataset
  num_classes = len(class_labels_to_int)
  ### model
  model_name = 'maxvit_large_tf_224'
  model_checkpoint = CELL_PAINTING_DIR + '/MaxVitModel_ep0.6706739526411658.pth'
  # optimizer_checkpoint = CELL_PAINTING_DIR + '/optimizer.pt'
  pretrained=True
  batch_size = 16
  num_epochs = 30
  num_workers = 20

  ### set only one to True
  save_best_loss = False
  save_best_accuracy = True

  optimizer = 'adamw' # ["rmsprop", "adam"]
  learning_rate = 5e-5
  adam_epsilon = 1e-6
  weight_decay = 1e-8 # for adamw
  l2_penalty = 0.01 # for RMSprop
  rms_momentum = 0 # for RMSprop

  ### learning rate scheduler (LRS)
  scheduler = 'ReduceLROnPlateau' # []

  # scheduler = 'CosineAnnealingLR'
  plateau_factor = 0.5
  plateau_patience = 3
  cosine_T_max = 4
  cosine_eta_min = 1e-8
  verbose = True

  ### train and validation DataLoaders
  shuffle = True
  random_seed = 42

  output_dir = MODEL_OUTPUT_DIR + '/' + str(date.today())
  checkpoint_last = output_dir + '/' + model_name + '/checkpoint-last'
  checkpoint_best = output_dir + '/' + model_name + '/checkpoint-best'

In [ ]:
os.environ['WANDB_API_KEY']='808606b1ec54e09c37c9c19ea6bb8d5b8a679987'

class WandBLogger(object):
    def __init__(self, variant, project, prefix=''):
      """
      Args:
        variant: dictionary of hyperparameters
        project: name of project
      """
      log_dir = tempfile.mkdtemp()
      if prefix != '':
          project = '{}--{}'.format(prefix, project)

      wandb.init(
          config=variant,
          project=project,
          dir=log_dir,
          id=uuid.uuid4().hex,
      )

    def log(self, *args, **kwargs):
      wandb.log(*args, **kwargs)

wblogger = WandBLogger(
    variant={
      'initial_learning_rate': CFG.learning_rate,
      'adam_epsilon': CFG.adam_epsilon,
      'num_epochs': CFG.num_epochs,
      'batch_size': CFG.batch_size
    },
    project=f'cellvit-{CFG.model_name}',
    prefix='Can-c037_27k'
)

In [ ]:
from maxvit_dataset import MaxVitDataset

In [ ]:
class MaxVitClassifier(nn.Module):
    def __init__(self, cfg, checkpoint=None):
        super().__init__()
        self.model_name = cfg.model_name
        self.model = timm.create_model(cfg.model_name, 
                                       pretrained=cfg.pretrained, 
                                       num_classes=cfg.num_classes)

    def forward(self, x):
        x = self.model(x)
        return x
    
    def freeze(self):
        # To freeze the residual layers
        for param in self.model.parameters():
            param.requires_grad = False

        for param in self.model.head.parameters():
            param.requires_grad = True
    
    def unfreeze(self):
        # Unfreeze all layers
        for param in self.model.parameters():
            param.requires_grad = True

In [ ]:
# Data augmentation and normalization for training
# Just normalization for validation

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'dev': transforms.Compose([
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

image_datasets = {x: MaxVitDataset(CFG, split=x,
                                   transform=data_transforms[x])
                  for x in ['train', 'dev']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], 
                                              batch_size=CFG.batch_size,
                                              num_workers=CFG.num_workers,
                                              shuffle=True)
              for x in ['train', 'dev']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'dev']}
# class_names = image_datasets['train'].classes
print(f"Dataset sizes: {dataset_sizes}")

device = torch.device("cuda:0" if torch.cuda.is_available() else CFG.device)
print(device)

In [ ]:
import random
def set_seed(cfg):
    random.seed(cfg.random_seed)
    np.random.seed(cfg.random_seed)
    torch.manual_seed(cfg.random_seed)
    if cfg.n_gpu > 0:
        torch.cuda.manual_seed_all(cfg.random_seed)

def train_model(cfg, model, dataloaders, criterion, optimizer):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0


    last_checkpoint_path = CFG.checkpoint_last
    last_scheduler_path = os.path.join(last_checkpoint_path, 'scheduler.pt')
    last_optimizer_path = os.path.join(last_checkpoint_path, 'optimizer.pt')
    best_checkpoint_path = CFG.checkpoint_best
    best_scheduler_path = os.path.join(best_checkpoint_path, 'scheduler.pt')
    best_optimizer_path = os.path.join(best_checkpoint_path, 'optimizer.pt')

    for epoch in range(cfg.num_epochs):
        print('Epoch {}/{}'.format(epoch, cfg.num_epochs - 1))
        print('-' * 10)

        wblogdict = {}

        # Each epoch has a training and validation phase
        for phase in ['train', 'dev']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            wblogdict[f'{phase}/loss'] = np.round(epoch_loss, 4)
            wblogdict[f'{phase}/acc'] = np.round(epoch_acc.cpu(), 4)

            if phase == "train":
              wblogdict['train/learning_rate'] = CFG.learning_rate

            if not os.path.exists(last_checkpoint_path):
                os.makedirs(last_checkpoint_path)
            
            # torch.save(model.state_dict(), last_checkpoint_path + f"/MaxVitModel_ep{epoch_acc}.pth")
            # torch.save(optimizer.state_dict(), last_optimizer_path)

            # deep copy the model
            if phase == 'dev' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            
                if not os.path.exists(best_checkpoint_path):
                    os.makedirs(best_checkpoint_path)

                torch.save(model.state_dict(), best_checkpoint_path + f"/MaxVitModel_ep{best_acc}.pth")
                torch.save(optimizer.state_dict(), best_optimizer_path)
  
            if phase == 'dev':
                val_acc_history.append(epoch_acc)
                # scheduler.step(epoch_loss)

        wblogger.log(wblogdict)
        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [ ]:
# model_ft = models.resnet18(pretrained=True)
# num_ftrs = model_ft.fc.in_features
# # Here the size of each output sample is set to 2.
# # Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
# model_ft.fc = nn.Linear(num_ftrs, 2)

set_seed(CFG)

checkpoint = CELL_PAINTING_DIR + '/MaxVitModel_ep0.6706739526411658.pth'
model_ft = MaxVitClassifier(CFG, checkpoint=checkpoint)
model_ft = model_ft.to(device)

params_to_update = model_ft.parameters()
# print("Params to learn:")

# for name,param in model_ft.named_parameters():
#     if param.requires_grad == True:
#             print("\t",name)

# Observe that all parameters are being optimized
# optimizer_ft = optim.SGD(model_ft.parameters(), lr=5e-5, momentum=0.9)

optimizer_ft = AdamW(model_ft.parameters(), lr=CFG.learning_rate, eps=CFG.adam_epsilon, weight_decay=CFG.weight_decay)

# scheduler = lr_scheduler.ReduceLROnPlateau(optimizer_ft, factor=CFG.plateau_factor, patience=CFG.plateau_patience)
criterion = nn.CrossEntropyLoss()

In [ ]:
model_ft, hist = train_model(CFG, model_ft, dataloaders, criterion, optimizer_ft)

In [ ]:
from sklearn.metrics import f1_score, confusion_matrix, ConfusionMatrixDisplay

def evaluate(cfg, model, dataloaders):
    y_true, y_pred = [], []
    model.eval()

    with torch.set_grad_enabled(False):
        for inputs, labels in tqdm(dataloaders["dev"]):
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            y_true += labels.tolist()
            y_pred += preds.tolist()

    f1 = f1_score(y_true, y_pred, average='weighted')

    print(f"F1 score: {f1}")
    return y_true, y_pred, f1

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else CFG.device)
print(device)

checkpoint = "/dev/shm/2023-04-09/maxvit_large_tf_224/checkpoint-best/MaxVitModel_ep0.7303339882121807.pth"
model_ft = MaxVitClassifier(CFG)
model_ft.load_state_dict(torch.load(checkpoint))
model_ft.to(device)
model_ft.eval()

In [ ]:
y_true, y_pred, f1 = evaluate(CFG, model_ft, dataloaders)

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [ ]:
class_labels = list(class_labels_to_int.keys())
cm = confusion_matrix(y_true, y_pred)

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_labels)
disp.plot()

plt.xticks(rotation=90)
# Pad margins so that markers don't get clipped by the axes
plt.margins(0.2)
# Tweak spacing to prevent clipping of tick-labels
plt.subplots_adjust(bottom=0.15)
plt.tight_layout()
plt.show()

In [ ]:
import seaborn as sns
import pandas as pd

sns.set_style("whitegrid")
sns.color_palette("pastel")

class_labels = list(class_labels_to_int.keys())
cm = confusion_matrix(y_true, y_pred)

df_cm = pd.DataFrame(cm, index = class_labels, columns = class_labels)
plt.figure(figsize = (10,7))
sns.heatmap(df_cm, annot=True, fmt=".1f", cmap="flare")